In [ ]:
from google.colab import files
jason=files.upload()

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c r-street-quant-challenge
!mkdir data
!unzip r-street-quant-challenge.zip -d data
!unzip test_data.csv.zip
!unzip train_data.csv.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
train_data.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
train_target.csv: Skipping, found more recently modified local copy (use --force to force download)
test_data.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
mkdir: cannot create directory ‘data’: File exists
unzip:  cannot find or open r-street-quant-challenge.zip, r-street-quant-challenge.zip.zip or r-street-quant-challenge.zip.ZIP.
Archive:  test_data.csv.zip
replace test_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
df=pd.read_csv("/content/train_data.csv")
df2=pd.read_csv("/content/test_data.csv")
df3=pd.read_csv("/content/train_target.csv")

In [ ]:
def adx(DF,n=20):
    "function to calculate ADX"
    df2 = DF.copy()
    df2['H-L']=abs(df2['high']-df2['low'])
    df2['H-PC']=abs(df2['high']-df2['close'].shift(1))
    df2['L-PC']=abs(df2['low']-df2['close'].shift(1))
    df2['TR']=df2[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df2['+DM']=np.where((df2['high']-df2['high'].shift(1))>(df2['low'].shift(1)-df2['low']),df2['high']-df2['high'].shift(1),0)
    df2['+DM']=np.where(df2['+DM']<0,0,df2['+DM'])
    df2['-DM']=np.where((df2['low'].shift(1)-df2['low'])>(df2['high']-df2['high'].shift(1)),df2['low'].shift(1)-df2['low'],0)
    df2['-DM']=np.where(df2['-DM']<0,0,df2['-DM'])

    df2["+DMMA"]=df2['+DM'].ewm(span=n,min_periods=n).mean()
    df2["-DMMA"]=df2['-DM'].ewm(span=n,min_periods=n).mean()
    df2["TRMA"]=df2['TR'].ewm(span=n,min_periods=n).mean()

    df2["+DI"]=100*(df2["+DMMA"]/df2["TRMA"])
    df2["-DI"]=100*(df2["-DMMA"]/df2["TRMA"])
    df2["DX"]=100*(abs(df2["+DI"]-df2["-DI"])/(df2["+DI"]+df2["-DI"]))
    
    df2["ADX"]=df2["DX"].ewm(span=n,min_periods=n).mean()

    return df2

In [ ]:
import math
df2["technical wonly"]=0.5
for i in range(20):
  df2["technical wonly"][i]=0.5
for j in range(20,10316):  
  if (df2['RSI_7'][j]>60 and df2['+DI'][j]>df2['-DI'][j]):
    df2["technical wonly"][j]=1
  elif df2['RSI_7'][j]<50 and df2['+DI'][j]>df2['-DI'][j]:
    df2["technical wonly"][j]=0
  

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(df["technical wonly"], df[''])

In [ ]:
difference=df2["technical wonly"]-df3['target']
difference.mean()